In [ ]:
f_func <- "./functions.ipynb"
eval(parse(text = system2('jupyter', c('nbconvert', f_func, '--to=script', '--stdout'), stdout = TRUE)))

f_data_loading <- "./data_loading.ipynb"
eval(parse(text = system2('jupyter', c('nbconvert', f_data_loading, '--to=script', '--stdout'), stdout = TRUE)))

In [ ]:
Afum_endXX <- Afum_endXX %>%
    mutate(taxName = "Aspergillus fumigatus")  # Adding the source label

QCreads_endXX <- QCreads_endXX %>%
    mutate(taxName = "QCreads")  # Adding the source label

combined_data <- bind_rows(Afum_endX, QCreads_endX) %>% select(-c("nsCount", "patient")) %>% 
    filter(EndXX == "CG") %>% 
    pivot_wider(names_from = taxName, values_from = sCount, values_fill = 0) 

Asp_species_count <- combined_data %>% 
    filter(read == "R1") %>% 
    mutate("db" = "EPRSc2") %>% 
    mutate("threshold" = "0.4") %>%
    rename("count" = "Aspergillus fumigatus") %>%
    mutate(taxName = "Aspergillus fumigatus") %>% 
    mutate(sample = sample_id) %>% 
    ungroup() %>% 
    select("taxName", "count", "db", "threshold", "sample")

tmp_QCreads <- QCreads_endXX %>% 
    filter(EndXX == "CG") %>%
    filter(read == "R1") %>% 
    ungroup() %>% 
    group_by(sample_id) %>% 
    summarize(QC_number_reads = sum(sCount))

total_read_count <- data.frame("EPRSc2" = tmp_QCreads$QC_number_reads, 
                               row.names = tmp_QCreads$sample_id) %>% t()

In [ ]:
Bctrls <- c('H01Bctrl','H05Bctrl','H06Bctrl','H07Bctrl','H08Bctrl','H10Bctrl','H11Bctrl','H22Bctrl','H24Bctrl')
Pctrls <- c('H31Pctrl','H32Pctrl','H33Pctrl','H34Pctrl','H35Pctrl','H36Pctrl','H37Pctrl','H38Pctrl','H39Pctrl')

ctrl_list <- list(Bctrls, Pctrls)
names(ctrl_list) <- c("Bctrls", "Pctrls")

In [ ]:
dbs_sel_min <- c('EPRSc2')
lst_fisher_species <- list()
for (fav_db in dbs_sel_min){
    if (fav_db == "EPRSc2"){print(fav_db); tmp_species_threshold = "conf0.4"}
    
    df_fisher_species <- Asp_species_count %>% 
        filter(db == fav_db) %>% 
        filter(threshold == gsub(tmp_species_threshold, pattern = "conf", replacement = "")) %>% 
        filter(grepl(sample, pattern = "asp$") | 
               grepl(sample, pattern = "ctrl$") | 
               grepl(sample, pattern = "ctrl2$") | 
               grepl(sample, pattern = "ctrl3$") | 
               grepl(sample, pattern = "VAL$")
              ) %>% 
        mutate(control_type = ifelse(grepl(sample, pattern = "Bctrl") | grepl(sample, pattern = "Basp"), 
                                     yes = "Bctrls", no = 
                              ifelse(grepl(sample, pattern = "Pctrl") | grepl(sample, pattern = "Pasp"), 
                                     yes = "Pctrls", no = NA))) %>% 
        mutate(origin = ifelse(control_type == "Bctrls", yes = "BAL ss-cfDNA", no = 
                        ifelse(control_type == "Pctrls", yes = "Plasma ss-cfDNA", no = NA))) %>% 
        mutate(fisher.t.ext = NA) %>%  mutate(fisher.t.int = NA) 

    for (i in 1:nrow(df_fisher_species)){
        db_tmp <- df_fisher_species$db[i]
        ctrl <- df_fisher_species$control_type[i]
        tn <- df_fisher_species$taxName[i]
        smpl <- df_fisher_species$sample[i]

        tmp_ext <- c()
        for (ext_c in ctrl_list[[ctrl]]){
            if(ext_c == c(df_fisher_species$sample[i])){next}
            ext_ctrl_species_count <- Asp_species_count %>% 
                filter(sample == ext_c) %>% 
                filter(db == db_tmp) %>% 
                filter(taxName == tn) %>% 
                filter(threshold == gsub(tmp_species_threshold, pattern = "conf", replacement = "")) %>% 
                pull("count")

            ft_ext <- as.numeric(fisher.test(
                            rbind(c(df_fisher_species$count[i],
                                    (total_read_count[db_tmp, smpl]-df_fisher_species$count[i])),
                                  c(ext_ctrl_species_count, 
                                    (total_read_count[db_tmp, ext_c])-ext_ctrl_species_count)), 
                            alternative="greater")$p.value)
            tmp_ext <- c(tmp_ext, ft_ext)
            rm(ft_ext)
        }
        df_fisher_species$fisher.t.ext[i] <- mean(tmp_ext, na.rm = TRUE)

        if(ctrl == "Pctrls" & df_fisher_species$sample[i] %nin% c('A05Pctrl2','A05Pctrl3')){
            int_c <- gsub(df_fisher_species$sample[i], pattern = "asp", replacement = "ctrl")
            if(int_c %in% Asp_species_count$sample & int_c %in% colnames(total_read_count)){
                int_ctrl_species_count <- Asp_species_count %>% 
                    filter(sample == ext_c) %>% 
                    filter(db == db_tmp) %>% 
                    filter(taxName == tn) %>% 
                    filter(threshold == gsub(tmp_species_threshold, pattern = "conf", replacement = "")) %>% 
                    pull("count")

                df_fisher_species$fisher.t.int[i] <- as.numeric(fisher.test(
                                rbind(c(df_fisher_species$count[i],
                                        (total_read_count[db_tmp, smpl]-df_fisher_species$count[i])),
                                      c(int_ctrl_species_count, 
                                        (total_read_count[db_tmp, int_c])-int_ctrl_species_count)), 
                                alternative="greater")$p.value)
            }
        }
    }
    lst_fisher_species[[fav_db]] <- df_fisher_species
}

In [ ]:
lst_fisher_species$EPRSc2 %>% filter(fisher.t.ext < 0.001) %>% 
    select(c("sample", "origin", "count", "fisher.t.ext")) %>% head(n = 20)